In [12]:
import pandas as pd
import sqlparse
import re
from collections import defaultdict
from sql_metadata import Parser
import pprint

from column_usage import get_column_usage_from_logs, parse_simple_logs, get_where_usage
from index_generation import generate_all_indexes, prune_indexes, generate_create_index_commands

# (i) Epinions

### Scale factor = 1000
```
 Schema |     Name     | Type  |    Owner     | Persistence | Access method |  Size   | Description 
--------+--------------+-------+--------------+-------------+---------------+---------+-------------
 public | observations | table | project1user | permanent   | heap          | 5746 MB | 
 public | sessions     | table | project1user | permanent   | heap          | 784 kB  | 
 public | sources      | table | project1user | permanent   | heap          | 208 kB  | 
 public | types        | table | project1user | permanent   | heap          | 1464 kB | 
```

### Without index
```
{
 "scalefactor": "100",
 "Current Timestamp (milliseconds)": 1645652423106,
 "Benchmark Type": "epinions",
 "isolation": "TRANSACTION_READ_COMMITTED",
 "DBMS Version": "PostgreSQL 14.2 (Ubuntu 14.2-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit",
 "Goodput (requests/second)": 19.59045481317814,
 "terminals": "1",
 "DBMS Type": "POSTGRES",
 "Latency Distribution": {
  "95th Percentile Latency (microseconds)": 161233,
  "Maximum Latency (microseconds)": 165597,
  "Median Latency (microseconds)": 47476,
  "Minimum Latency (microseconds)": 10717,
  "25th Percentile Latency (microseconds)": 19026,
  "90th Percentile Latency (microseconds)": 77681,
  "99th Percentile Latency (microseconds)": 162937,
  "75th Percentile Latency (microseconds)": 57150,
  "Average Latency (microseconds)": 51774
 },
 "Throughput (requests/second)": 19.262581092455495
}
```

In [13]:
log_file_path = "data/epinions_scale_100_rate_unlimited_time_60_simple.csv"
log_file_dataframe = pd.read_csv(log_file_path, header=None)
epinions_parsing_success, epinions_column_usage, epinions_where_predicates = parse_simple_logs(log_file_dataframe)

Total log lines:  3791


In [14]:
pprint.pprint(epinions_parsing_success)

{'failure': 7,
 'failure_reason': defaultdict(<function parse_simple_logs.<locals>.<lambda> at 0x11c13cee0>, {}),
 'skipped': 2450,
 'skipped_log_type': defaultdict(<function parse_simple_logs.<locals>.<lambda> at 0x11c13ce50>, {}),
 'success': 1314,
 'success_log_type': defaultdict(<function parse_simple_logs.<locals>.<lambda> at 0x11c13c8b0>, {})}


In [15]:
pprint.pprint(epinions_column_usage)

defaultdict(<function parse_simple_logs.<locals>.<lambda> at 0x11c13c670>,
            {'order_by': defaultdict(<function parse_simple_logs.<locals>.<lambda>.<locals>.<lambda> at 0x11bf7df70>,
                                     {'rating': 239,
                                      'review.creation_date': 356}),
             'select': defaultdict(<function parse_simple_logs.<locals>.<lambda>.<locals>.<lambda> at 0x11b792160>,
                                   {'*': 239,
                                    'item.i_id': 1,
                                    'rating': 110,
                                    'review.*': 117,
                                    'review.rating': 121,
                                    'trust.*': 117,
                                    'useracct.u_id': 1}),
             'update': defaultdict(<function parse_simple_logs.<locals>.<lambda>.<locals>.<lambda> at 0x11bf7de50>,
                                   {'item.title': 124,
                            

In [16]:
pprint.pprint(epinions_where_predicates)

defaultdict(<function parse_simple_logs.<locals>.<lambda> at 0x11c13c940>,
            {(('item.i_id', None),): 124,
             (('item.i_id', 'review.i_id'), ('review.i_id', None)): 128,
             (('review.i_id', None),): 238,
             (('review.i_id', None), ('review.u_id', None)): 117,
             (('review.u_id', 'trust.target_u_id'), ('review.i_id', None), ('trust.source_u_id', None)): 110,
             (('trust.source_u_id', None),): 117,
             (('trust.source_u_id', None), ('trust.target_u_id', None)): 250,
             (('useracct.u_id', None),): 117,
             (('useracct.u_id', 'review.u_id'), ('review.u_id', None)): 111})


### Adding single indices:

`CREATE INDEX idx_rewview_iid ON review (i_id);`

`CREATE INDEX idx_trust_sid ON trust (source_u_id);`

`CREATE INDEX idx_trust_tid ON trust (target_u_id);`

`CREATE INDEX idx_review_uid ON review (u_id);`

`CREATE INDEX idx_item_iid ON item (i_id);`

`CREATE INDEX idx_useracct_uid ON useracct (u_id);`


```
{
 "scalefactor": "100",
 "Current Timestamp (milliseconds)": 1645655337931,
 "Benchmark Type": "epinions",
 "isolation": "TRANSACTION_READ_COMMITTED",
 "DBMS Version": "PostgreSQL 14.2 (Ubuntu 14.2-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit",
 "Goodput (requests/second)": 1997.9683018947728,
 "terminals": "1",
 "DBMS Type": "POSTGRES",
 "Latency Distribution": {
  "95th Percentile Latency (microseconds)": 1211,
  "Maximum Latency (microseconds)": 34752,
  "Median Latency (microseconds)": 322,
  "Minimum Latency (microseconds)": 109,
  "25th Percentile Latency (microseconds)": 222,
  "90th Percentile Latency (microseconds)": 1069,
  "99th Percentile Latency (microseconds)": 1859,
  "75th Percentile Latency (microseconds)": 739,
  "Average Latency (microseconds)": 505
 },
 "Throughput (requests/second)": 1969.9516633065623
}
```

In [18]:
generate_create_index_commands(prune_indexes(generate_all_indexes(epinions_column_usage, epinions_where_predicates)))

['CREATE INDEX idx_trust_source_u_id_target_u_id ON trust (source_u_id, target_u_id)',
 'CREATE INDEX idx_review_u_id ON review (u_id)',
 'CREATE INDEX idx_review_i_id_u_id ON review (i_id, u_id)',
 'CREATE INDEX idx_item_i_id ON item (i_id)',
 'CREATE INDEX idx_useracct_u_id ON useracct (u_id)']

### Adding multi indices:

`CREATE INDEX idx_trust_comb ON trust (source_u_id, target_u_id);`

`CREATE INDEX idx_review_comb ON review (i_id, u_id);`

`CREATE INDEX idx_item_iid ON item (i_id);`

`CREATE INDEX idx_useracct_uid ON useracct (u_id);`

`CREATE INDEX idx_review_uid ON review (u_id);`

`CREATE INDEX idx_trust_tid ON trust (target_u_id);`

```
{
 "scalefactor": "100",
 "Current Timestamp (milliseconds)": 1645657598501,
 "Benchmark Type": "epinions",
 "isolation": "TRANSACTION_READ_COMMITTED",
 "DBMS Version": "PostgreSQL 14.2 (Ubuntu 14.2-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit",
 "Goodput (requests/second)": 2024.5618091032316,
 "terminals": "1",
 "DBMS Type": "POSTGRES",
 "Latency Distribution": {
  "95th Percentile Latency (microseconds)": 1226,
  "Maximum Latency (microseconds)": 35793,
  "Median Latency (microseconds)": 312,
  "Minimum Latency (microseconds)": 105,
  "25th Percentile Latency (microseconds)": 211,
  "90th Percentile Latency (microseconds)": 1070,
  "99th Percentile Latency (microseconds)": 1902,
  "75th Percentile Latency (microseconds)": 733,
  "Average Latency (microseconds)": 499
 },
 "Throughput (requests/second)": 1991.4621610037952
}
```

# (ii) Jungle

### Scale factor = 1000
```
 Schema |     Name      | Type  |    Owner     | Persistence | Access method |  Size   | Description 
--------+---------------+-------+--------------+-------------+---------------+---------+-------------
 public | jungle        | table | project1user | permanent   | heap          | 6011 MB | 
```

### Without index

```
{
    "scalefactor": "1000",
    "Current Timestamp (milliseconds)": 1645468671728,
    "Benchmark Type": "indexjungle",
    "isolation": null,
    "DBMS Version": "PostgreSQL 14.2 (Ubuntu 14.2-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit",
    "Goodput (requests/second)": 1.6112963871000103,
    "terminals": "1",
    "DBMS Type": "POSTGRES",
    "Latency Distribution": {
        "95th Percentile Latency (microseconds)": 632282,
        "Maximum Latency (microseconds)": 646268,
        "Median Latency (microseconds)": 623239,
        "Minimum Latency (microseconds)": 607371,
        "25th Percentile Latency (microseconds)": 619141,
        "90th Percentile Latency (microseconds)": 630341,
        "99th Percentile Latency (microseconds)": 644008,
        "75th Percentile Latency (microseconds)": 627854,
        "Average Latency (microseconds)": 623411
    },
    "Throughput (requests/second)": 1.601329605324134
}
```

In [ ]:
log_file_path = "./data/jungle_scale_1000_rate_unlimited_time_600_simple.csv"
log_file_dataframe = pd.read_csv(log_file_path, header=None)
jungle_parsing_success, jungle_column_usage, jungle_where_predicates = parse_simple_logs(log_file_dataframe)

In [ ]:
pprint.pprint(jungle_parsing_success)

In [ ]:
pprint.pprint(jungle_column_usage)

In [ ]:
pprint.pprint(jungle_where_predicates)

In [ ]:
generate_create_index_commands(prune_indexes(generate_all_indexes(jungle_column_usage, jungle_where_predicates)))

### Added index: 

`CREATE INDEX idx_jungle_int_field3 ON jungle (int_field3);`
```
{
 "scalefactor": "1000",
 "Current Timestamp (milliseconds)": 1645469508485,
 "Benchmark Type": "indexjungle",
 "isolation": null,
 "DBMS Version": "PostgreSQL 14.2 (Ubuntu 14.2-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit",
 "Goodput (requests/second)": 1383.0397270709402,
 "terminals": "1",
 "DBMS Type": "POSTGRES",
 "Latency Distribution": {
  "95th Percentile Latency (microseconds)": 1251,
  "Maximum Latency (microseconds)": 7005,
  "Median Latency (microseconds)": 711,
  "Minimum Latency (microseconds)": 150,
  "25th Percentile Latency (microseconds)": 435,
  "90th Percentile Latency (microseconds)": 1173,
  "99th Percentile Latency (microseconds)": 1408,
  "75th Percentile Latency (microseconds)": 997,
  "Average Latency (microseconds)": 722
 },
 "Throughput (requests/second)": 1382.1797272406525
}
```

# (iii) Timeseries

### Scale factor = 10
```
 Schema |     Name     | Type  |    Owner     | Persistence | Access method |  Size   | Description 
--------+--------------+-------+--------------+-------------+---------------+---------+-------------
 public | observations | table | project1user | permanent   | heap          | 5746 MB | 
 public | sessions     | table | project1user | permanent   | heap          | 784 kB  | 
 public | sources      | table | project1user | permanent   | heap          | 208 kB  | 
 public | types        | table | project1user | permanent   | heap          | 1464 kB | 
```

### Without index
```
{
 "scalefactor": "10",
 "Current Timestamp (milliseconds)": 1645546729938,
 "Benchmark Type": "timeseries",
 "isolation": null,
 "DBMS Version": "PostgreSQL 14.2 (Ubuntu 14.2-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit",
 "Goodput (requests/second)": 0.591362724923002,
 "terminals": "1",
 "DBMS Type": "POSTGRES",
 "Latency Distribution": {
  "95th Percentile Latency (microseconds)": 1884720,
  "Maximum Latency (microseconds)": 1931072,
  "Median Latency (microseconds)": 1687350,
  "Minimum Latency (microseconds)": 1553243,
  "25th Percentile Latency (microseconds)": 1659923,
  "90th Percentile Latency (microseconds)": 1845196,
  "99th Percentile Latency (microseconds)": 1921746,
  "75th Percentile Latency (microseconds)": 1726973,
  "Average Latency (microseconds)": 1705219
 },
 "Throughput (requests/second)": 0.5847181999238672
}
```

In [7]:
log_file_path = "./data/timeseries_scale_10_rate_unlimited_time_300_simple.csv"
log_file_dataframe = pd.read_csv(log_file_path, header=None)
timeseries_parsing_success, timeseries_column_usage, timeseries_where_predicates = parse_simple_logs(log_file_dataframe)


Total log lines:  616


In [8]:
pprint.pprint(timeseries_parsing_success)

{'failure': 6,
 'failure_reason': defaultdict(<function parse_simple_logs.<locals>.<lambda> at 0x11c13c9d0>, {}),
 'skipped': 412,
 'skipped_log_type': defaultdict(<function parse_simple_logs.<locals>.<lambda> at 0x11c13c040>, {}),
 'success': 180,
 'success_log_type': defaultdict(<function parse_simple_logs.<locals>.<lambda> at 0x11c13c3a0>, {})}


In [9]:
pprint.pprint(timeseries_column_usage)

defaultdict(<function parse_simple_logs.<locals>.<lambda> at 0x11bf7d670>,
            {'order_by': defaultdict(<function parse_simple_logs.<locals>.<lambda>.<locals>.<lambda> at 0x11b5ceca0>,
                                     {'observations.created_time': 180}),
             'select': defaultdict(<function parse_simple_logs.<locals>.<lambda>.<locals>.<lambda> at 0x11b5ce9d0>,
                                   {'observations.*': 180}),
             'where': defaultdict(<function parse_simple_logs.<locals>.<lambda>.<locals>.<lambda> at 0x11b5ceee0>,
                                  {'observations.session_id': 180,
                                   'observations.source_id': 180,
                                   'observations.type_id': 180})})


In [10]:
pprint.pprint(timeseries_where_predicates)

defaultdict(<function parse_simple_logs.<locals>.<lambda> at 0x11b5cef70>,
            {(('observations.source_id', None), ('observations.session_id', None), ('observations.session_id', None), ('observations.type_id', None)): 180})


### Adding single indices:

`CREATE INDEX idx_observations_created_time ON observations (created_time);`

`CREATE INDEX idx_observations_session_id ON observations (session_id);`

`CREATE INDEX idx_observations_source_id ON observations (source_id);`

`CREATE INDEX idx_observations_type_id ON observations (type_id);`

```
{
 "scalefactor": "10",
 "Current Timestamp (milliseconds)": 1645555654179,
 "Benchmark Type": "timeseries",
 "isolation": null,
 "DBMS Version": "PostgreSQL 14.2 (Ubuntu 14.2-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit",
 "Goodput (requests/second)": 25.843852375660493,
 "terminals": "1",
 "DBMS Type": "POSTGRES",
 "Latency Distribution": {
  "95th Percentile Latency (microseconds)": 66818,
  "Maximum Latency (microseconds)": 86139,
  "Median Latency (microseconds)": 41596,
  "Minimum Latency (microseconds)": 4061,
  "25th Percentile Latency (microseconds)": 29577,
  "90th Percentile Latency (microseconds)": 63509,
  "99th Percentile Latency (microseconds)": 74320,
  "75th Percentile Latency (microseconds)": 53250,
  "Average Latency (microseconds)": 38786
 },
 "Throughput (requests/second)": 25.764118160849353
}
```

In [11]:
generate_create_index_commands(prune_indexes(generate_all_indexes(timeseries_column_usage, timeseries_where_predicates)))


['CREATE INDEX idx_observations_session_id_source_id_type_id ON observations (session_id, source_id, type_id)']

### Adding multiindex:

`CREATE INDEX idx_observations_comb ON observations (source_id, session_id, type_id);`

```
{
 "scalefactor": "10",
 "Current Timestamp (milliseconds)": 1645556187951,
 "Benchmark Type": "timeseries",
 "isolation": null,
 "DBMS Version": "PostgreSQL 14.2 (Ubuntu 14.2-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit",
 "Goodput (requests/second)": 1514.628448292875,
 "terminals": "1",
 "DBMS Type": "POSTGRES",
 "Latency Distribution": {
  "95th Percentile Latency (microseconds)": 1157,
  "Maximum Latency (microseconds)": 5221,
  "Median Latency (microseconds)": 607,
  "Minimum Latency (microseconds)": 230,
  "25th Percentile Latency (microseconds)": 459,
  "90th Percentile Latency (microseconds)": 1038,
  "99th Percentile Latency (microseconds)": 1346,
  "75th Percentile Latency (microseconds)": 812,
  "Average Latency (microseconds)": 660
 },
 "Throughput (requests/second)": 1512.1035304469679
}
```

### Adding index on sorted field:

\+ `CREATE INDEX idx_observations_created_time ON observations (created_time);`

```
{
 "scalefactor": "10",
 "Current Timestamp (milliseconds)": 1645556588675,
 "Benchmark Type": "timeseries",
 "isolation": null,
 "DBMS Version": "PostgreSQL 14.2 (Ubuntu 14.2-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit",
 "Goodput (requests/second)": 1482.5532255022954,
 "terminals": "1",
 "DBMS Type": "POSTGRES",
 "Latency Distribution": {
  "95th Percentile Latency (microseconds)": 1176,
  "Maximum Latency (microseconds)": 4890,
  "Median Latency (microseconds)": 620,
  "Minimum Latency (microseconds)": 236,
  "25th Percentile Latency (microseconds)": 473,
  "90th Percentile Latency (microseconds)": 1056,
  "99th Percentile Latency (microseconds)": 1371,
  "75th Percentile Latency (microseconds)": 826,
  "Average Latency (microseconds)": 674
 },
 "Throughput (requests/second)": 1480.0398923517653
}
```

# TODOs:

- Some columns do not have table prefixed (rating vs review.rating)
- Fix column usage; is it even required?